<a href="https://colab.research.google.com/github/specM7/DSGP_Group_33_Brain_Tumor_Predictor/blob/Meningioma_Adrian_2425482/meningioma_adrian_2425482.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI-Powered Brain Tumor Predictor for Meningioma(Adrian)


## Project Goal
Build a Convolutional Neural Network (CNN) to classify brain MRI images into 4 tumor categories.

## Technologies Used
- **TensorFlow/Keras** - Deep Learning Framework
- **OpenCV** - Image Processing
- **NumPy** - Numerical Computing


## Model Specifications
- **Input**: 224x224 grayscale MRI images
- **Architecture**: Custom CNN with 3 convolutional blocks
- **Output**: 4-class classification (tumor types)
- **Training**: 25 epochs with validation split

###  Objectives
1. Build a robust CNN model for brain tumor classification
2. Achieve >90% accuracy on test data
3. Provide interpretable results with visualization

In [1]:
import os, glob, cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


##  Extract Dataset
Unzips your file and shows folder structure.

---

## 2️ Dataset Preparation

###  Upload Dataset
Upload your `DataSet.zip` file containing:
```
DataSet/
├── Training/
│   ├── glioma/
│   ├── meningioma/
│   ├── notumor/
│   └── pituitary/
└── Testing/
    ├── glioma/
    ├── meningioma/
    ├── notumor/
    └── pituitary/
```

**Click the upload button below to select your dataset ZIP file.**

In [2]:
from google.colab import files

uploaded = files.upload()        # PICK the real DataSet.zip from your PC

zip_name = list(uploaded.keys())[0]
print("Uploaded:", zip_name)

# Unzip into /content/DataSet_raw
!rm -rf /content/DataSet_raw
!mkdir -p /content/DataSet_raw
!unzip -q "$zip_name" -d /content/DataSet_raw

print("\nTop level under /content/DataSet_raw:")
!ls /content/DataSet_raw

print("\nAny 'Training' folders:")
!find /content/DataSet_raw -maxdepth 5 -type d -iname "Training"


Saving DataSet.zip to DataSet.zip
Uploaded: DataSet.zip

Top level under /content/DataSet_raw:
DataSet

Any 'Training' folders:
/content/DataSet_raw/DataSet/Training


##  Set Paths
Look at previous output, find your Training folder path, and paste it in next cell.

Example: `/content/DataSet_raw/DataSet/Training`

In [3]:
# ===============================
# Imports
# ===============================
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ===============================
# Paths
# ===============================
training_path = "/content/DataSet_raw/DataSet/Training"  # adjust ONLY if needed
raw_root = os.path.dirname(training_path)

print("training_path:", training_path)
print("raw_root:", raw_root)
print("Training folders:", os.listdir(training_path))

train_dir = training_path
test_dir = os.path.join(raw_root, "Testing")

print("train_dir:", train_dir)
print("test_dir:", test_dir)

# ===============================
# Parameters
# ===============================
img_size = (224, 224)
batch_size = 32

# ===============================
# Image Generators
# ===============================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.9, 1.1],
    horizontal_flip=True,
    validation_split=0.2
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# ===============================
# Datasets
# ===============================
train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    subset='training',
    seed=42
)

val_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    subset='validation',
    seed=42
)

test_ds = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    shuffle=False,
    seed=42
)

# ===============================
# Class Info
# ===============================
print("Class indices:", train_ds.class_indices)


training_path: /content/DataSet_raw/DataSet/Training
raw_root: /content/DataSet_raw/DataSet
Training folders: ['glioma', 'pituitary', 'meningioma', 'notumor']
train_dir: /content/DataSet_raw/DataSet/Training
test_dir: /content/DataSet_raw/DataSet/Testing
Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
Found 1826 images belonging to 4 classes.
Class indices: {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


## 6️ Model Evaluation

###  Testing on Unseen Data

Now we'll evaluate the model on completely unseen test data to measure real-world performance.

**Test Set:** Independent images never seen during training

In [4]:
model = models.Sequential([
    layers.Input(shape=(*img_size, 1)),
    layers.Rescaling(1./255),

    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.Recall(class_id=1, name='meningioma_recall'),
        tf.keras.metrics.Precision(class_id=1, name='meningioma_precision')
    ]
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25
)

test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    25,690,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,084,484 (99.50 MB)

 Trainable params: 26,082,500 (99.50 MB)

 Non-trainable params: 1,984 (7.75 KB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 1141s 8s/step - accuracy: 0.6298 - loss: 1.1960 - meningioma_precision: 0.7722 - meningioma_recall: 0.1879 - val_accuracy: 0.2550 - val_loss: 9.8901 - val_meningioma_precision: 0.0000e+00 - val_meningioma_recall: 0.0000e+00
Epoch 2/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 1144s 8s/step - accuracy: 0.7389 - loss: 0.7043 - meningioma_precision: 0.7500 - meningioma_recall: 0.2061 - val_accuracy: 0.2550 - val_loss: 25.9929 - val_meningioma_precision: 0.0000e+00 - val_meningioma_recall: 0.0000e+00
Epoch 3/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 1131s 8s/step - accuracy: 0.7746 - loss: 0.5898 - meningioma_precision: 0.7508 - meningioma_recall: 0.1895 - val_accuracy: 0.2550 - val_loss: 45.1119 - val_meningioma_precision: 0.0000e+00 - val_meningioma_recall: 0.0000e+00
Epoch 4/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 1155s 8s/step - accuracy: 0.7981 - loss: 0.5437 - meningioma_precision: 0.7997 - meningioma_recall: 0.1878 - val_accuracy: 0.2550 - val_loss: 49.8219 - val_meningioma

ValueError: too many values to unpack (expected 2)

##  Save & Export Model

###  Saving Trained Model

The model will be saved in HDF5 format (.h5) which includes:
- ✅ Model architecture
- ✅ Trained weights
- ✅ Optimizer state
- ✅ Training configuration

**File:** `brain_tumor_mri_model.h5`

In [ ]:
model.save("brain_tumor_mri_model.h5")


In [ ]:
from google.colab import files
files.download("brain_tumor_mri_model.h5")
